In [3]:
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp

Populating the interactive namespace from numpy and matplotlib


In [5]:
p_df = pd.read_csv('250_apps_with_desc.csv')
docs = array(p_df['description'])


In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs

In [7]:
docs = docs_preprocessor(docs)

In [8]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [9]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 7575
Number of unique words after removing rare and common words: 784


In [10]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [11]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 784
Number of documents: 250


In [12]:
from gensim.models import LdaModel

In [13]:
# Set training parameters.
num_topics = 4
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


CPU times: user 11.2 s, sys: 11.7 ms, total: 11.2 s
Wall time: 12.4 s


In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [15]:
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/srv/conda/envs/notebook/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.016305 -0.088534       1        1  32.959114
0      0.188926  0.037937       2        1  25.510088
1     -0.065338 -0.075881       3        1  22.297665
2     -0.107282  0.126478       4        1  19.233139, topic_info=    Category        Freq             Term       Total  loglift  logprob
185  Default  250.000000             food  250.000000  30.0000  30.0000
563  Default  202.000000           recipe  202.000000  29.0000  29.0000
160  Default  234.000000            train  234.000000  28.0000  28.0000
524  Default  229.000000          booking  229.000000  27.0000  27.0000
445  Default  240.000000  online_shopping  240.000000  26.0000  26.0000
700  Default  186.000000            hotel  186.000000  25.0000  25.0000
336  Default  176.000000           ticket  176.000000  24.0000  24.0000
758  Default  135.000000       order_food  135.000000  23.0000  23.0000
528  Default  157.000000           flight  157.000000  22.0000  22.0000
250  Default  128.000000            pizza  128.000000  21.0000  21.0000
253  Default  139.000000       restaurant  139.000000  20.0000  20.0000
757  Default  102.000000    food_delivery  102.000000  19.0000  19.0000
333  Default  148.000000         property  148.000000  18.0000  18.0000
429  Default  123.000000          fashion  123.000000  17.0000  17.0000
496  Default   98.000000      real_estate   98.000000  16.0000  16.0000
640  Default   93.000000     train_ticket   93.000000  15.0000  15.0000
701  Default   81.000000   indian_railway   81.000000  14.0000  14.0000
635  Default   81.000000      irctc_train   81.000000  13.0000  13.0000
85   Default  110.000000           status  110.000000  12.0000  12.0000
189  Default   71.000000  fruit_vegetable   71.000000  11.0000  11.0000
287  Default   87.000000           design   87.000000  10.0000  10.0000
376  Default   87.000000           reward   87.000000   9.0000   9.0000
387  Default   70.000000             trip   70.000000   8.0000   8.0000
22   Default  118.000000      credit_card  118.000000   7.0000   7.0000
530  Default   65.000000            irctc   65.000000   6.0000   6.0000
202  Default   56.000000             meat   56.000000   5.0000   5.0000
225  Default  146.000000    your_doorstep  146.000000   4.0000   4.0000
388  Default   77.000000          vehicle   77.000000   3.0000   3.0000
152  Default   96.000000             earn   96.000000   2.0000   2.0000
119  Default  118.000000            house  118.000000   1.0000   1.0000
..       ...         ...              ...         ...      ...      ...
596   Topic4   10.989558             late   14.429061   1.3762  -6.2490
231   Topic4   21.468090        delicious   28.195145   1.3760  -5.5793
553   Topic4   13.199041            drink   17.404613   1.3719  -6.0658
673   Topic4   40.392136         calendar   56.070782   1.3206  -4.9473
338   Topic4    8.339168         beverage   11.679567   1.3117  -6.5249
520   Topic4   13.085640            those   18.392912   1.3081  -6.0744
550   Topic4   17.732590             cook   25.266106   1.2945  -5.7705
217   Topic4    9.553762            south   13.643936   1.2922  -6.3890
748   Topic4   28.193928             near   42.607986   1.2356  -5.3068
258   Topic4   14.679481            taste   22.439022   1.2242  -5.9595
237   Topic4   13.467188           jaipur   21.404373   1.1852  -6.0457
178   Topic4   22.941399          deliver   37.101360   1.1678  -5.5130
232   Topic4    8.870347       delivering   14.639456   1.1475  -6.4632
591   Topic4   21.556883         business   36.658810   1.1176  -5.5752
199   Topic4   23.709602          kitchen   43.303398   1.0462  -5.4800
409   Topic4   52.257492             room  118.298676   0.8315  -4.6897
226   Topic4   27.921370     across_india   56.426994   0.9450  -5.3165
99    Topic4   58.396187    your_favorite  180.439850   0.5204  -4.5787
177   Topic4   32.888950         

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

p_df['tokenz'] = docs

docs1 = p_df['tokenz'].apply(lambda l: l[:int0(len(l)/2)])
docs2 = p_df['tokenz'].apply(lambda l: l[int0(len(l)/2):])


In [18]:
corpus1 = [dictionary.doc2bow(doc) for doc in docs1]
corpus2 = [dictionary.doc2bow(doc) for doc in docs2]

# Using the corpus LDA model tranformation
lda_corpus1 = model[corpus1]
lda_corpus2 = model[corpus2]

In [19]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=False):
    
    '''
    LDA transformation, for each doc only returns topics with non-zero weight
    This function makes a matrix transformation of docs in the topic space.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]

    return array(top_dist), keys

In [22]:
top_dist1, _ = get_doc_topic_dist(model, lda_corpus1)
top_dist2, _ = get_doc_topic_dist(model, lda_corpus2)

print("Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):")
print(mean([cosine_similarity(c1.reshape(1, -1), c2.reshape(1, -1))[0][0] for c1,c2 in zip(top_dist1, top_dist2)]))

random_pairs = np.random.randint(0, len(p_df['description']), size=(400, 2))

print("Inter similarity: cosine similarity between random parts (lower is better):")
print(np.mean([cosine_similarity(top_dist1[i[0]].reshape(1, -1), top_dist2[i[1]].reshape(1, -1)) for i in random_pairs]))

Intra similarity: cosine similarity for corresponding parts of a doc(higher is better):
0.87929493
Inter similarity: cosine similarity between random parts (lower is better):
0.6982814


In [23]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [24]:
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print

term                 frequency

Topic 0 |---------------------

train                0.031
booking              0.030
hotel                0.024
ticket               0.023
flight               0.021
status               0.013
credit_card          0.012
train_ticket         0.012
indian_railway       0.011
irctc_train          0.011
Topic 1 |---------------------

property             0.019
real_estate          0.014
house                0.012
your_doorstep        0.011
fruit_vegetable      0.011
fresh                0.010
real                 0.010
http                 0.009
meat                 0.009
grocery              0.008
Topic 2 |---------------------

food                 0.043
recipe               0.035
order_food           0.024
restaurant           0.023
pizza                0.022
food_delivery        0.018
design               0.012
your_favorite        0.010
room                 0.009
your_doorstep        0.009
Topic 3 |---------------------

online_shopping      0.025
fas

In [25]:
top_labels = {0: 'Travel', 1:'estates', 2:'food and beverages', 3:'Technology and Beauty'}

In [38]:
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

def paper_to_wordlist( paper, remove_stopwords=True ):
    '''
        Function converts text to a sequence of words,
        Returns a list of words.
    '''
    lemmatizer = WordNetLemmatizer()
    # 1. Remove non-letters
    paper_text = re.sub("[^a-zA-Z]"," ", paper)
    # 2. Convert words to lower case and split them
    words = paper_text.lower().split()
    # 3. Remove stop words
    words = [w for w in words if not w in stops]
    # 4. Remove short words
    words = [t for t in words if len(t) > 2]
    # 5. lemmatizing
    words = [nltk.stem.WordNetLemmatizer().lemmatize(t) for t in words]

    return(words)


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvectorizer = TfidfVectorizer(input='content', analyzer = 'word', lowercase=True, stop_words='english',\
                                  tokenizer=paper_to_wordlist, ngram_range=(1, 3), min_df=40, max_df=0.20,\
                                  norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

dtm = tvectorizer.fit_transform(p_df['description']).toarray()



/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [42]:
top_dist =[]
for d in corpus:
    tmp = {i:0 for i in range(num_topics)}
    tmp.update(dict(model[d]))
    vals = list(OrderedDict(tmp).values())
    top_dist += [array(vals)]

In [43]:
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)
features = tvectorizer.get_feature_names()

In [44]:
top_ws = []
for n in range(len(dtm)):
    inds = int0(argsort(dtm[n])[::-1][:4])
    tmp = [features[i] for i in inds]
    
    top_ws += [' '.join(tmp)]
    
p_df['Text_Rep'] = pd.DataFrame(top_ws)
p_df['clusters'] = pd.DataFrame(lda_keys)
p_df['clusters'].fillna(10, inplace=True)

cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

p_df['colors'] = p_df['clusters'].apply(lambda l: cluster_colors[l])

In [45]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

In [46]:
p_df['X_tsne'] =X_tsne[:, 0]
p_df['Y_tsne'] =X_tsne[:, 1]

In [47]:
from bokeh.plotting import figure, show, output_notebook, save#, output_file
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [51]:
source = ColumnDataSource(dict(
    x=p_df['X_tsne'],
    y=p_df['Y_tsne'],
    color=p_df['colors'],
    label=p_df['clusters'].apply(lambda l: top_labels[l]),
#     msize= p_df['marker_size'],
    topic_key= p_df['clusters'],
    title= p_df[u'title'],
    content = p_df['Text_Rep']
))

In [58]:
title = 'Clustering of 250 topics'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover,save",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source,
                 color='color', alpha=0.8, size=10)#'msize', )

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "title: @title, KeyWords: @content - Topic: @topic_key "}
plot_lda.legend.location = "top_left"

show(plot_lda)

#save the plot
# save(plot_lda, '{}.html'.format(title))